In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from Utils.data_handler import load_images

In [2]:
data_dir = 'sample_Stereo_conveyor_without_occlusions_distorted/left'
image_array = load_images(data_dir)

Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_489372015_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_522562027_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_555752039_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_588932037_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_622132063_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_655312061_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_688502073_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_721692085_Left.png
Reading Image: ./../Data/sample_Stereo_conveyor_without_occlusions_distorted/left/1585434283_754872084_L

## Sparse Optical Flow

In [3]:
frame0 = image_array[0]
gray0 = cv2.cvtColor(frame0, cv2.COLOR_RGB2GRAY)
mask = np.zeros_like(frame0)# store the former lines
# define a feature extractor
#cl = cv2.xfeatures2d_SIFT.create()
cl = cv2.ORB_create()

# switch
optical = True #opitcal flow
orbon = not optical #matcher

# crop the Object
objectMask = np.zeros_like(gray0)
roi = cv2.selectROI('gray', gray0) # select the first object I wanna track in the image
x,y,w,h = roi
objectMask[y:y+h, x:x+w] = 1 #the object in the left
if optical:
    feat0 = cv2.goodFeaturesToTrack(gray0, mask = objectMask, maxCorners=100, qualityLevel=0.2, minDistance=2, blockSize=7)
if orbon:
    kps1, des1 = cl.detectAndCompute(gray0, objectMask)
#roi = cv2.selectROI('gray', gray0) # select the first object I wanna track in the image
#x,y,w,h = roi
#objectMask[y:y+h, x:x+w] = 1 #the object in the left
#if optical:
#    feat2 = cv2.goodFeaturesToTrack(gray0, mask = 

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [6]:
for frame_num in list(range(1,len(image_array))):
    frame = image_array[frame_num] # read a frame
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    feat1, status, error = cv2.calcOpticalFlowPyrLK(gray0, gray, feat0, None)
    #feat3, status, error = cv2.calcOpticalFlowPyrLK(gray0, gray, feat2, None)
    #mask = frame

    for i in range(len(feat1)):
    
        f10=int(feat0[i][0][0])
        f11=int(feat0[i][0][1])
        f20=int(feat1[i][0][0])
        f21=int(feat1[i][0][1])

        cv2.line(mask, (f10,f11), (f20, f21), (0, 255, 0), 2)
        cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)
        cv2.circle(frame, (f10, f11), 5, (0, 0, 255), -1)
    
        # cv2.line(mask, (feat0[i][0][0], feat0[i][0][1]), (feat1[i][0][0], feat1[i][0][1]), (0, 255, 0), 2)
        # cv2.circle(frame, (feat0[i][0][0], feat0[i][0][1]), 5, (255, 0, 0), -1)
        # cv2.circle(frame, (feat1[i][0][0], feat1[i][0][1]), 5, (0, 0, 255), -1)

    #for i in range(len(feat3)):
    #    cv2.line(mask, (feat2[i][0][0], feat2[i][0][1]), (feat3[i][0][0], feat3[i][0][1]), (255, 0, 0), 2)
    #    cv2.circle(frame, (feat2[i][0][0], feat2[i][0][1]), 5, (255, 0, 0), -1)
    #    cv2.circle(frame, (feat3[i][0][0], feat3[i][0][1]), 5, (0, 0, 255), -1)
    while (1):
        cv2.imshow('feature track', cv2.add(frame, mask))
        k = cv2.waitKey(1)
        break
    feat0 = feat1
    gray0 = gray

cv2. destroyAllWindows() 

## Dense Optical Flow

In [7]:
# # Get a VideoCapture object from video and store it in vс
# vc = cv2.VideoCapture(“Car.mp4”)
# # Read first frame
# _, first_frame = vc.read()
first_frame = image_array[0]
# Scale and resize image
resize_dim = 600
max_dim = max(first_frame.shape)
scale = resize_dim/max_dim
first_frame = cv2.resize(first_frame, None, fx=scale, fy=scale)
# Convert to gray scale 
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

In [8]:
# Create mask
mask = np.zeros_like(first_frame)
# Set image saturation to maximum value as we do not need it
mask[:,:, 1] = 255

In [ ]:
for frame_num in list(range(1,len(image_array))):
    frame = image_array[frame_num] # read a frame
    # Convert new frame format`s to gray scale and resize gray frame obtained
    gray = cv2.resize(frame, None, fx=scale, fy=scale)
    gray = cv2.resize(gray, None, fx=scale, fy=scale)

    # Calculate dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[:,:, 0], flow[:,:, 1])
    # Set image hue according to the optical flow direction
    mask[:,:, 0] = angle * 180 / np.pi / 2
    # Set image value according to the optical flow magnitude (normalized)
    mask[:,:, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convert HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

    # Resize frame size to match dimensions    
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    # Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
    cv2.imshow("Dense optical flow", dense_flow)
    # Update previous frame
    prev_gray = gray
    # Frame are read by intervals of 10 millisecond. The programs breaks out of the while loop when the user presses the ‘q’ key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()